In [1]:
print('Inicializando Variáveis do Programa')

#inicializando variáveis

variável_dependente = 'qty_2019'
variáveis_independentes = ('qty_2014', 'porcent_pagos')
ano_início = 2014
best_r2=False
pareamento=True
capes=False
cnpq=True
nenhum=True
ambos=False
preenche_todos_ids = True
verbose = True

Inicializando Variáveis do Programa


In [2]:
from datetime import datetime
print('Início em: ', datetime.now())

print('Importações Iniciais')

from tkinter import NONE
from Lattes import Lattes
from Carga import Carga
from Database import Database
import psycopg2, pickle
import pandas, os
import statsmodels.api as sm
from urllib.parse import urlencode
from psycopg2.extensions import AsIs
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import Database
import json
import locale
import statsmodels.formula.api as sm2
from itertools import combinations




print('Fim: ', datetime.now())

Início em:  2023-04-24 10:36:00.129687
Importações Iniciais
Fim:  2023-04-24 10:36:08.182992


In [3]:
sql = '''
select * from financiamentos_doutorandos_2014
    left join indicadores_doutorado
        on indicadores_doutorado.id = financiamentos_doutorandos_2014.id
    inner join indicadores_nomes
        on indicadores_nomes.tipo = indicadores_doutorado.indicador_tipo                 
'''

In [4]:
#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

df = pd.read_sql(sql, engine)
if verbose: print(f'df.size: {df.size}')

print('Término em: ', datetime.now())

Carregando indicadores na memória. Início em:  2023-04-24 10:36:08.287723
df.size: 16139320
Término em:  2023-04-24 10:36:27.795568


In [5]:
sql_ids_pareados = '''
select pareado_1 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_2 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_3 as id from public.par_doutorandos_2014_sem_reposicao
'''

In [6]:
print('Carregando indicadores dos pareados na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

pareados = pd.read_sql(sql_ids_pareados, engine)
if verbose: print(f'pareados.size: {pareados.size}')

print('Término em: ', datetime.now())

Carregando indicadores dos pareados na memória. Início em:  2023-04-24 10:36:28.072830
pareados.size: 2419
Término em:  2023-04-24 10:36:28.417903


In [7]:
## Normalizando as Tabelas

print('Realizando normalização da tabela. Início em: ', datetime.now())

df['const'] = 1
df = df.loc[:,~df.columns.duplicated()]

df['bool_cnpq'] = df['pagtos'].map(lambda pagtos: 1 if pagtos > 0 else 0)
df['bool_capes'] = df['capes'].map(lambda capes: 1 if capes == True else 0)



#Preenchendo Nan com zeros nos indicadores
df.qty_2012 = df.qty_2012.fillna(0)
df.qty_2013 = df.qty_2013.fillna(0)
df.qty_2014 = df.qty_2014.fillna(0)
df.qty_2015 = df.qty_2015.fillna(0)
df.qty_2016 = df.qty_2016.fillna(0)
df.qty_2017 = df.qty_2017.fillna(0)
df.qty_2018 = df.qty_2018.fillna(0)
df.qty_2019 = df.qty_2019.fillna(0)
df.capes = df.capes.fillna(0)
df.pagtos = df.pagtos.fillna(0)

valor_total_cnpq =  df[['id', 'pagtos']].loc[df['pagtos']>0].drop_duplicates().sum().pagtos
df['porcent_pagos'] = df['pagtos']/valor_total_cnpq


print('Fim em: ', datetime.now())


Realizando normalização da tabela. Início em:  2023-04-24 10:36:28.460832
Fim em:  2023-04-24 10:36:30.472272


In [8]:
df['grupo'] = df.apply(lambda d: 'Ambos' if (d['capes'] == True and d['pagtos']>0) \
                       else 'CAPES' if (d['capes'] == True and d['pagtos']==0) \
                       else 'CNPq' if d['pagtos'] > 0 \
                       else 'Nenhum', axis=1)
dt = df

In [9]:
#Apagando os financiados 
if not capes:
    if verbose: print('Dropping CAPES')
    dt.drop(dt.loc[dt.grupo == 'CAPES'].index, inplace=True)
if not cnpq:
    if verbose: print('Dropping CNPq')
    dt.drop(dt.loc[dt.grupo == 'CNPq'].index, inplace=True)
if not nenhum:
    if verbose: print('Dropping no financed')
    dt.drop(dt.loc[dt.grupo == 'Nenhum'].index, inplace=True)
if not ambos:
    if verbose: print('Dropping Ambos')
    dt.drop(dt.loc[dt.grupo == 'Ambos'].index, inplace=True)

#Apagando os que não foram escolhidos pelo pareamento
if pareamento:   
    if verbose: print('Dropping Não Pareados')
    dt.drop(dt.loc[~dt.id.isin(pareados.id)].loc[dt.pagtos==0].index, inplace=True)

Dropping CAPES
Dropping Ambos
Dropping Não Pareados


In [10]:
tempo_início = datetime.now()
tempo_último = datetime.now()
feitos = 0
def print_time(feitos, número_total, tempo_início, tempo_último, atual):
    if not feitos == 0:
        porcentagem_já_feita = (feitos/número_total)
        print_percent = '{0:.{1}f}'.format(porcentagem_já_feita*100, 1)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / feitos
        tempo_restante = (número_total - feitos) * tempo_por_id
        tempo_em_que_vai_acabar = (datetime.now() + tempo_restante).strftime("%d/%m/%y %H:%M:%S")
        print(f'{feitos}/{número_total}. {print_percent}% feitos. Fazendo: {atual}. Acabará em {tempo_em_que_vai_acabar}. O Último demorou: {datetime.now() - tempo_último}.')
    else:
        print(f'{feitos}/{número_total}. Fazendo: {atual}.')
#para chamar a função:   
#    print_time(feitos, número_total, tempo_início, tempo_último, atual)
#    feitos += 1
#    tempo_último = datetime.now()

In [11]:
def regressão(X,y):
    try:
        # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
        reg = sm.OLS(y, X).fit()
        # mostrando as estatísticas do modelo
        #reg.summary()
        
        
        
        #guardando a regressão na memória

        result = {
                'Indicador': indicador,
                'Erro': False,
                'Prob (F-statistic)':reg.f_pvalue, 
                'Parâmetros': reg.params, 
                "P>|t|":reg.pvalues, 
                'Standard Error': reg.bse, 
                "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                'Number of observations n.': reg.nobs, 
                "R2 ajustado": reg.rsquared_adj, 
                "Modelo":reg.model,
            }
    except:
        result = {
                'Indicador': indicador,
                'Erro': True,
                'Prob (F-statistic)':None, 
                'Parâmetros': None, 
                "P>|t|":None, 
                'Standard Error': None, 
                "Covariância entre os pasrâmetros": None, 
                'Number of observations n.': None, 
                "R2 ajustado": None, 
                "Modelo": None,
            }
    return result

In [12]:
def regressão_r2(dados, variável_dependente, variáveis_independentes):
    
    lista_formulas = []
    
    if best_r2:
        for x in range(1,len(variáveis_independentes)+1):
            for a in combinations(variáveis_independentes,x):
                formula = f'{variável_dependente} ~ ' 
                if not a[0] == '-1':
                    formula = (formula + ' + '.join(a)).replace('+ -1',' -1')
                    lista_formulas.append(formula)
    else:
        lista_formulas.append(variável_dependente + ' ~ ' + ' + '.join(variáveis_independentes).replace('+ -1',' -1'))
        
    if verbose: print(lista_formulas)
    
    list_results = []
    for formula in lista_formulas:
    
        try:
            # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
            reg_ajustado = sm2.ols(formula, data = dados)
            reg = reg_ajustado.fit()
            # mostrando as estatísticas do modelo
            if verbose: print('\nCom constante:\n',reg.summary2())  

            #guardando a regressão na memória

            result = {
                    'Indicador': indicador,
                    'Erro': False,
                    'Prob (F-statistic)':reg.f_pvalue, 
                    'Parâmetros': reg.params, 
                    "P>|t|":reg.pvalues, 
                    'Standard Error': reg.bse, 
                    "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                    'Number of observations n.': reg.nobs, 
                    "R2 ajustado": reg.rsquared_adj, 
                    "Modelo":reg.model,                }
        except:
            result = {
                    'Indicador': indicador,
                    'Erro': True,
                    'Prob (F-statistic)':None, 
                    'Parâmetros': None, 
                    "P>|t|":None, 
                    'Standard Error': None, 
                    "Covariância entre os pasrâmetros": None, 
                    'Number of observations n.': None, 
                    "R2 ajustado": None, 
                    "Modelo": None,                }
    
        list_results.append(result)
        
    maiorR = max(list_results, key=lambda x:x['R2 ajustado'])
        
    return maiorR

In [13]:
def regressão_com_confiança (X,y,confiança = 0.05): 
    result = regressão(X,y)
    if verbose: print(result)
    while result['Erro'] == False and result['P>|t|'].max() > confiança:
        X = X.drop(columns=result['P>|t|'].idxmax())
        result = regressão(X,y)
        if verbose: print(result)
        
        
    if result['Erro'] == False and result['Prob (F-statistic)'] < confiança and result['P>|t|'].max() < confiança:
        result['Confiança'] = confiança
        return result
    else:
        result['Confiança'] = 'Erro'
    return result


In [14]:
def preenche_data_frame_com_zeros(dados, indicador = 'Artigo Publicado', verbose=verbose):

    dados = df.loc[df.nome == indicador]
    lista_ids = df.loc[~df.id.isin(dados.id)].id.unique()
    feitos = 0
    tempo_início = datetime.now()
    tempo_último = datetime.now()

    if verbose: 
        número_total = len(lista_ids)
        num_feitos = 0
        print('\nPreenchendo com zeros os Ids faltantes. Tamanho: ', número_total)

    for id in lista_ids:
        if verbose: 
            atual = id
            print_time(feitos, número_total, tempo_início, tempo_último, atual)
            feitos += 1
            tempo_último = datetime.now()
        try:
            indicador_tipo = df.loc[df.nome == indicador].first_valid_index.indicador_tipo.mode()[0]
        except:
            indicador_tipo = None
        try:
            indicador_grupo = df.loc[df.nome == indicador].indicador_grupo.mode()[0]
        except:
            indicador_grupo = None            

        row = {
            'id': id,
            'pagtos': 0,
            'capes': False,
            'indicador_tipo': indicador_tipo,
            'indicador_grupo': indicador_grupo,
            'indicador': indicador,
            'qty_2012': 0,
            'qty_2013': 0,
            'qty_2014': 0,
            'qty_2015': 0,
            'qty_2016': 0,
            'qty_2017': 0,
            'qty_2018': 0,
            'qty_2019': 0,
            'tipo': indicador_tipo,
            'nome': indicador,
            'grupo': 'Nenhum',
            'path': None,
            'const': 1,
            'bool_cnpq': 0,
            'bool_capes': 0,
            'porcent_pagos': 0
                }
        dados = dados.append(row, ignore_index=True)

    if verbose: print('Terminado.\n')
    return dados
        

indicador = 'Artigo Publicado'
dados = dt.loc[dt.nome==indicador]
print(len(dados))
if preenche_todos_ids: 
    dados = preenche_data_frame_com_zeros()
print('Tamanho de dados: ', len(dados.id.unique()))
result = regressão_r2(dados, variável_dependente, variáveis_independentes = ('qty_2014', 'porcent_pagos', '-1'))
print(result)

In [ ]:
#inicializando variáveis
lista_regressões = []

print(f'Iniciado em {tempo_início}.')

#pegando lista de todos os indicadores
todos_indicadores = dt.nome.unique()

#iterando sobre cada indicador
número_total = len(todos_indicadores)
for indicador in todos_indicadores:
    
    #Realizando previsão de término
    print_time(feitos, número_total, tempo_início, tempo_último, indicador)
    feitos += 1
    tempo_último = datetime.now()

    #Realizando a regressão
    dados = dt.loc[dt.nome==indicador]

    if preenche_todos_ids: 
        dados = preenche_data_frame_com_zeros(dados, indicador, verbose = False)
    result = regressão_r2(dados, variável_dependente, variáveis_independentes)

    lista_regressões.append(result)


Iniciado em 2023-04-24 10:36:56.095988.
0/128. Fazendo: Conclusão de Doutorado.
['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.018    
Dependent Variable: qty_2019         AIC:                3843.5373
Date:               2023-04-24 10:37 BIC:                3862.9889
No. Observations:   4836             Log-Likelihood:     -1918.8  
Df Model:           2                F-statistic:        44.06    
Df Residuals:       4833             Prob (F-statistic): 1.09e-19 
R-squared:          0.018            Scale:              0.12955  
------------------------------------------------------------------
                Coef.   Std.Err.    t     P>|t|   [0.025   0.975] 
------------------------------------------------------------------
Intercept        0.8794   0.0062 141.8057 0.0000   0.8672   0.8915
qty_2014         0.3517   0.0931   3.7789 0.0002   0.1693   0.5342
porcent_pa

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.812     
Dependent Variable: qty_2019         AIC:                27240.9830
Date:               2023-04-24 10:43 BIC:                27260.4345
No. Observations:   4836             Log-Likelihood:     -13617.   
Df Model:           2                F-statistic:        1.044e+04 
Df Residuals:       4833             Prob (F-statistic): 0.00      
R-squared:          0.812            Scale:              16.354    
-------------------------------------------------------------------
                Coef.   Std.Err.    t     P>|t|   [0.025    0.975] 
-------------------------------------------------------------------
Intercept        0.5842   0.0754   7.7456 0.0000   0.4363    0.7320
qty_2014         1.2044   0.0084 143.0449 0.0000   1.1879    1.2209
porcent_pagos 1014.4194 226.2131   4.4844 0.0000 570.9387 1457.9000
---------

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.833     
Dependent Variable: qty_2019         AIC:                -6824.4024
Date:               2023-04-24 10:52 BIC:                -6804.9509
No. Observations:   4836             Log-Likelihood:     3415.2    
Df Model:           2                F-statistic:        1.209e+04 
Df Residuals:       4833             Prob (F-statistic): 0.00      
R-squared:          0.833            Scale:              0.014269  
--------------------------------------------------------------------
                 Coef.   Std.Err.     t      P>|t|   [0.025   0.975]
--------------------------------------------------------------------
Intercept        0.1276    0.0055   23.3121  0.0000  0.1169   0.1383
qty_2014         0.8731    0.0057  152.3335  0.0000  0.8619   0.8844
porcent_pagos   21.5506    6.4263    3.3535  0.0008  8.9522  34.1491
---

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.846    
Dependent Variable: qty_2019         AIC:                2898.3316
Date:               2023-04-24 11:13 BIC:                2917.7831
No. Observations:   4836             Log-Likelihood:     -1446.2  
Df Model:           2                F-statistic:        1.326e+04
Df Residuals:       4833             Prob (F-statistic): 0.00     
R-squared:          0.846            Scale:              0.10655  
------------------------------------------------------------------
                Coef.   Std.Err.    t     P>|t|   [0.025   0.975] 
------------------------------------------------------------------
Intercept        0.0460   0.0049   9.4318 0.0000   0.0365   0.0556
qty_2014         1.1026   0.0078 140.6656 0.0000   1.0873   1.1180
porcent_pagos  587.3047  31.5846  18.5947 0.0000 525.3845 649.2249
-----------------------

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                   Results: Ordinary least squares
Model:               OLS              Adj. R-squared:     0.716     
Dependent Variable:  qty_2019         AIC:                27212.2161
Date:                2023-04-24 11:35 BIC:                27231.6676
No. Observations:    4836             Log-Likelihood:     -13603.   
Df Model:            2                F-statistic:        6087.     
Df Residuals:        4833             Prob (F-statistic): 0.00      
R-squared:           0.716            Scale:              16.257    
--------------------------------------------------------------------
                Coef.   Std.Err.    t     P>|t|    [0.025    0.975] 
--------------------------------------------------------------------
Intercept        0.9477   0.1019   9.3023 0.0000    0.7480    1.1474
qty_2014         1.2868   0.0117 109.9859 0.0000    1.2638    1.3097
porcent_pagos 1608.6068 207.1386   7.7658 0.0000 1202.5208 2014.6

...

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.276     
Dependent Variable: qty_2019         AIC:                -7581.3932
Date:               2023-04-24 14:32 BIC:                -7561.9417
No. Observations:   4836             Log-Likelihood:     3793.7    
Df Model:           2                F-statistic:        920.8     
Df Residuals:       4833             Prob (F-statistic): 0.00      
R-squared:          0.276            Scale:              0.012201  
-------------------------------------------------------------------
                Coef.   Std.Err.    t    P>|t|    [0.025    0.975] 
-------------------------------------------------------------------
Intercept        0.0050   0.0016  3.1521 0.0016    0.0019    0.0082
qty_2014         1.1737   0.0786 14.9416 0.0000    1.0197    1.3277
porcent_pagos 1841.2079  47.8928 38.4444 0.0000 1747.3163 1935.0995
---------

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.921    
Dependent Variable: qty_2019         AIC:                8548.0559
Date:               2023-04-24 14:52 BIC:                8567.5074
No. Observations:   4836             Log-Likelihood:     -4271.0  
Df Model:           2                F-statistic:        2.819e+04
Df Residuals:       4833             Prob (F-statistic): 0.00     
R-squared:          0.921            Scale:              0.34270  
------------------------------------------------------------------
               Coef.   Std.Err.    t     P>|t|   [0.025    0.975] 
------------------------------------------------------------------
Intercept       0.0228   0.0085   2.6789 0.0074   0.0061    0.0394
qty_2014        1.0357   0.0045 230.1248 0.0000   1.0269    1.0445
porcent_pagos 712.2137 161.7786   4.4024 0.0000 395.0541 1029.3734
-----------------------

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                   Results: Ordinary least squares
Model:                OLS              Adj. R-squared:     0.768    
Dependent Variable:   qty_2019         AIC:                9195.4771
Date:                 2023-04-24 15:12 BIC:                9214.9287
No. Observations:     4836             Log-Likelihood:     -4594.7  
Df Model:             2                F-statistic:        7997.    
Df Residuals:         4833             Prob (F-statistic): 0.00     
R-squared:            0.768            Scale:              0.39179  
--------------------------------------------------------------------
                Coef.   Std.Err.    t     P>|t|    [0.025    0.975] 
--------------------------------------------------------------------
Intercept        0.0541   0.0092   5.8482 0.0000    0.0359    0.0722
qty_2014         1.1275   0.0097 115.7338 0.0000    1.1084    1.1466
porcent_pagos 1385.8228  92.5748  14.9698 0.0000 1204.3340 1567.3

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                   Results: Ordinary least squares
Model:               OLS              Adj. R-squared:     0.423    
Dependent Variable:  qty_2019         AIC:                6160.6361
Date:                2023-04-24 15:31 BIC:                6180.0876
No. Observations:    4836             Log-Likelihood:     -3077.3  
Df Model:            2                F-statistic:        1775.    
Df Residuals:        4833             Prob (F-statistic): 0.00     
R-squared:           0.423            Scale:              0.20917  
-------------------------------------------------------------------
                Coef.   Std.Err.    t    P>|t|    [0.025    0.975] 
-------------------------------------------------------------------
Intercept        0.0327   0.0067  4.9016 0.0000    0.0196    0.0458
qty_2014         1.3161   0.0294 44.8037 0.0000    1.2585    1.3737
porcent_pagos 2609.5503  95.1987 27.4116 0.0000 2422.9175 2796.1831
---------

['qty_2019 ~ qty_2014 + porcent_pagos']

Com constante:
                       Results: Ordinary least squares
Model:                 OLS                Adj. R-squared:       1.000       
Dependent Variable:    qty_2019           AIC:                  -318740.1405
Date:                  2023-04-24 15:52   BIC:                  -318720.6890
No. Observations:      4836               Log-Likelihood:       1.5937e+05  
Df Model:              2                  F-statistic:          1.170e+32   
Df Residuals:          4833               Prob (F-statistic):   0.00        
R-squared:             1.000              Scale:                1.3898e-30  
----------------------------------------------------------------------------
               Coef.  Std.Err.           t            P>|t|   [0.025  0.975]
----------------------------------------------------------------------------
Intercept     -0.0000   0.0000               -16.1218 0.0000 -0.0000 -0.0000
qty_2014       1.0000   0.0000 13030131093

In [ ]:
regressões = pd.DataFrame(lista_regressões)
regressões = pd.concat([regressões, 
                        regressões['Parâmetros'].apply(pd.Series), 
                        regressões['P>|t|'].apply(pd.Series), 
                        regressões['Standard Error'].apply(pd.Series)], 
                       axis=1)
regressões.to_excel(f'Regressões dos Doutorandos {ano_início} com Pareamento {pareamento} e sem Reposição com preenchimento CNPq {cnpq} CAPES {capes} Ambos {ambos} Nenhum {nenhum}.xlsx')